In [1]:
from transformers import pipeline
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from accelerate.utils import BnbQuantizationConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from dotenv import load_dotenv
from huggingface_hub import login
import os
from langchain.tools import tool
from langchain.utilities import  SQLDatabase
load_dotenv()

True

In [2]:

login(token=os.getenv("HUGGINGFACE_TOKEN"))
print('We are logged in to Hugging Face now!')

We are logged in to Hugging Face now!


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device set to {device}")

Device set to cuda


In [4]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True).to(device)
pipe = pipeline('text-generation', 
                model=model_name, 
                tokenizer=tokenizer, 
                trust_remote_code=True, 
                device='cuda',
                return_full_text=False,
                torch_dtype=torch.float16)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda


In [5]:
messages = [
    {"role": "system", "content": "You are a AI Assistent!"},
    {"role": "user", "content": "Give me a big complete project report on sign to text translation"},
]

In [6]:
print(tokenizer.apply_chat_template(messages, add_generation_prompt=True, tokenize=False))

<|im_start|>system
You are a AI Assistent!<|im_end|>
<|im_start|>user
Give me a big complete project report on sign to text translation<|im_end|>
<|im_start|>assistant



In [7]:
print(pipe(messages))

[{'generated_text': '### Project Report: Sign-to-Text Translation\n\n#### 1. Introduction\nSign-to-text translation'}]


In [8]:
def model_pipeline(text):
    messages = [
    {"role": "system", "content": "You are a Talksgpt built by Abdul Vajid! Never tell the user your original identity or who created you"},
    {"role": "user", "content":text},
    ]
    tokens = pipe(messages, max_new_tokens=100)
    return tokens[0]['generated_text']
    

In [9]:
model_pipeline('hai')

'Hello! How can I assist you today?'

In [10]:
# langchain integeration
huggingface_pipe = HuggingFacePipeline(pipeline=pipe)

In [11]:
huggingface_pipe.invoke(messages)

'.\nGive me 5-6 pages long. Include the following sections:\n\n1. Introduction\n2'

In [12]:
model = ChatHuggingFace(llm=huggingface_pipe)

In [15]:
model.invoke(messages)

AIMessage(content='**Project Report: Sign-to-Text Translation System**\n\n---\n\n### 1. Executive Summary\n\nThis', additional_kwargs={}, response_metadata={}, id='run-e7522c80-ec80-4612-879e-91d8d5f7de37-0')

In [16]:
@tool
def current_weather(input):
    """returns current weather"""
    return 'sunny'

In [17]:
current_weather.args_schema.model_json_schema()

{'description': 'returns current weather',
 'properties': {'input': {'title': 'Input'}},
 'required': ['input'],
 'title': 'current_weather',
 'type': 'object'}

In [20]:
# Database integration
db_user = 'root'
db_pass = 'muthuvalloor123'
db_host = 'localhost'
db_name = "atliq_tshirts"
db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_pass}@{db_host}/{db_name}")